In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

In [3]:
repo_url = "https://github.com/aakashvardhan/s22-unet-vae.git"

local_dir = '/content/s22-unet-vae'



# Check if the local directory already exists
if not os.path.exists(local_dir):
    # Clone the repository because it does not exist
    !git clone {repo_url}
else:
    # Change directory to the local repository
    %cd {local_dir}
    # Pull the latest changes because the repository already exists
    !git pull

/content/s22-unet-vae
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 6 (delta 3), reused 6 (delta 3), pack-reused 0
Unpacking objects: 100% (6/6), 2.77 KiB | 257.00 KiB/s, done.
From https://github.com/aakashvardhan/s22-unet-vae
   14bbc54..d1dea02  main       -> origin/main
Updating 14bbc54..d1dea02
Fast-forward
 data/oxford_iiit.py                      | 2 +-
 notebooks/train-unet-model-1-colab.ipynb | 2 +-
 2 files changed, 2 insertions(+), 2 deletions(-)


In [4]:
%cd s22-unet-vae

[Errno 2] No such file or directory: 's22-unet-vae'
/content/s22-unet-vae


In [5]:
import sys

sys.path.append('.')
sys.path.append('./models')
sys.path.append('./data')
sys.path.append('./notebooks')
sys.path.append('./utils')

In [6]:
!pip install -q -r requirements.txt

In [7]:
from lightning.pytorch import Trainer, seed_everything
import lightning as pl
from lightning.pytorch.callbacks import (ModelCheckpoint,
                                         LearningRateMonitor,
                                         RichModelSummary,
                                         EarlyStopping)
from config import UNetConfig, load_config, update_config
import torch
import os
from lit_unet import LitUNet
from datamodule import DataModule
from callbacks import ClassAccuracyLoss, PlotExampleCallback
import wandb
from lightning.pytorch.loggers import WandbLogger

# os.environ["WANDB_NOTEBOOK_NAME"] = "./notebooks/train-unet-model-1.ipynb"
# wandb.init(settings=wandb.Settings(_service_wait=300))

In [8]:
torch.cuda.empty_cache()
pl.seed_everything(42, workers=True)

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


42

In [9]:
config = UNetConfig()
json_data = load_config("training_1.json")
config = update_config(config, json_data)
print(config)

UNetConfig(in_channels=3, out_channels=3, num_filters=64, num_layers=4, batch_size=16, learning_rate=0.001, epochs=25, root_dir='./data', height=240, width=240, optimizer='adam', loss_function='cross_entropy', channel_reduction_method='max_pool', channel_expansion_method='conv_transpose', softmax_dim=1, dropout_rate=0.5, augmentation=True)


In [10]:
config.batch_size = 32
print(config)

UNetConfig(in_channels=3, out_channels=3, num_filters=64, num_layers=4, batch_size=32, learning_rate=0.001, epochs=25, root_dir='./data', height=240, width=240, optimizer='adam', loss_function='cross_entropy', channel_reduction_method='max_pool', channel_expansion_method='conv_transpose', softmax_dim=1, dropout_rate=0.5, augmentation=True)


In [11]:
data_module = DataModule(config)
data_module.setup()

In [12]:
model = LitUNet(config)

In [13]:
trainer = pl.Trainer(fast_dev_run=True)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
INFO:lightning.pytorch.utilities.rank_zero:Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


In [14]:
trainer.fit(model=model, datamodule=data_module)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
INFO: 
  | Name          | Type               | Params | Mode 
-------------------------------------------------------------
0 | model         | UNet               | 121 K  | train
1 | train_acc     | MulticlassAccuracy | 0      | train
2 | valid_acc     | MulticlassAccuracy | 0      | train
3 | loss_function | CrossEntropyLoss   | 0      | train
-------------------------------------------------------------
121 K     Trainable params
0         Non-trainable params
121 K     Total params
0.486     Total estimated model p

Training: |          | 0/? [00:00<?, ?it/s]

KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = self.dataset.__getitems__(possibly_batched_index)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataset.py", line 419, in __getitems__
    return [self.dataset[self.indices[idx]] for idx in indices]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataset.py", line 419, in <listcomp>
    return [self.dataset[self.indices[idx]] for idx in indices]
  File "/content/s22-unet-vae/data/oxford_iiit.py", line 57, in __getitem__
    transformed_mask = transformed_mask["mask"]
KeyError: 'mask'
